Estimate left-pose of an image.
(Provide image from a dataset for a reference.)

In [ ]:
import numpy as np
import cv2

def detect_features(image):
    """Detect keypoints and descriptors in the image using ORB."""
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2):
    """Match features using BFMatcher."""
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def estimate_pose(matches, keypoints1, keypoints2, K):
    """Estimate the pose using solvePnP."""
    if len(matches) < 4:
        raise ValueError("Not enough matches to estimate pose.")

    # Extract matched keypoints
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])

    # Define corresponding 3D points (assuming some fixed depth or using a known pattern)
    # In practice, you would use actual 3D coordinates of the matched points
    obj_points = np.float32([[p[0], p[1], 0] for p in pts1])

    # Assume the camera matrix (intrinsic parameters)
    camera_matrix = K
    dist_coeffs = np.zeros(4)  # assuming no lens distortion

    success, rvec, tvec = cv2.solvePnP(obj_points, pts2, camera_matrix, dist_coeffs)

    if not success:
        raise ValueError("Pose estimation failed.")

    # Convert rotation vector to rotation matrix
    R, _ = cv2.Rodrigues(rvec)

    # Construct the pose matrix
    pose = np.eye(4)
    pose[:3, :3] = R
    pose[:3, 3] = tvec[:, 0]

    return pose

def get_left_pose(image_path):
    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")

    # Assume we have a reference image for matching
    reference_image = image

    # Detect features in both images
    kp1, des1 = detect_features(image)
    kp2, des2 = detect_features(reference_image)

    # Match features
    matches = match_features(des1, des2)

    # Assume a dummy camera intrinsic matrix for pose estimation
    K = np.array([[800, 0, image.shape[1]//2],
                  [0, 800, image.shape[0]//2],
                  [0, 0, 1]], dtype=np.float32)

    # Estimate pose
    left_pose = estimate_pose(matches, kp1, kp2, K)

    return left_pose

# Example usage:
image_path = '/content/stcg.us.0003_f8f1f2c0b57a4d9981ef602f99030def_cam-1_change-1.png'
left_pose = get_left_pose(image_path)
print("Left Pose:\n", left_pose)

Estimate right-pose of an image.
(Provide image from a dataset for a reference.)

In [ ]:
import numpy as np
import cv2

def detect_features(image):
    """Detect keypoints and descriptors in the image using ORB."""
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2):
    """Match features using BFMatcher."""
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def estimate_pose(matches, keypoints1, keypoints2, K):
    """Estimate the pose using solvePnP."""
    if len(matches) < 4:
        raise ValueError("Not enough matches to estimate pose.")

    # Extract matched keypoints
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])

    # Define corresponding 3D points (assuming some fixed depth or using a known pattern)
    # In practice, you would use actual 3D coordinates of the matched points
    obj_points = np.float32([[p[0], p[1], 0] for p in pts1])

    # Assume the camera matrix (intrinsic parameters)
    # For example purposes, we use a simple camera matrix
    camera_matrix = K
    dist_coeffs = np.zeros(4)  # assuming no lens distortion

    success, rvec, tvec = cv2.solvePnP(obj_points, pts2, camera_matrix, dist_coeffs)

    if not success:
        raise ValueError("Pose estimation failed.")

    # Convert rotation vector to rotation matrix
    R, _ = cv2.Rodrigues(rvec)

    # Construct the pose matrix
    pose = np.eye(4)
    pose[:3, :3] = R
    pose[:3, 3] = tvec[:, 0]

    return pose

def get_right_pose(image_path, reference_image_path):
    # Load the images
    image = cv2.imread(image_path)
    reference_image = cv2.imread(reference_image_path)

    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")

    if reference_image is None:
        raise ValueError(f"Reference image at path {reference_image_path} could not be loaded.")

    # Detect features in both images
    kp1, des1 = detect_features(image)
    kp2, des2 = detect_features(reference_image)

    # Match features
    matches = match_features(des1, des2)

    # Assume a dummy camera intrinsic matrix for pose estimation
    K = np.array([[800, 0, image.shape[1]//2],
                  [0, 800, image.shape[0]//2],
                  [0, 0, 1]], dtype=np.float32)

    # Estimate pose
    right_pose = estimate_pose(matches, kp1, kp2, K)

    return right_pose

# Example usage:
image_path = '/content/stcg.us.0003_f8f1f2c0b57a4d9981ef602f99030def_cam-1_change-10001.png'
reference_image_path = '/content/stcg.us.0003_f8f1f2c0b57a4d9981ef602f99030def_cam-1_change-1.png'
right_pose = get_right_pose(image_path, reference_image_path)
print("Right Pose:\n", right_pose)